In [0]:
import keras
import pandas as pd
import csv
import os
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import BayesianRidge, LinearRegression

In [0]:
dir_path = '/content/drive/My Drive/NASA - Integrated Assessment/cov data/split/'
header = ['iso_code','location','date','total_cases','new_cases','total_deaths','new_deaths','total_cases_per_million','new_cases_per_million','total_deaths_per_million','new_deaths_per_million','total_tests','new_tests','total_tests_per_thousand','new_tests_per_thousand','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_units','stringency_index','population','population_density','median_age','aged_65_older','aged_70_older','gdp_per_capita','extreme_poverty','cvd_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','hospital_beds_per_100k']

#os.listdir(dir_path)

In [0]:
drop_vs = ['iso_code', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_tests','new_tests','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_units','median_age','aged_70_older','cvd_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','extreme_poverty']
single_vs = ['population_density','aged_65_older','gdp_per_capita','hospital_beds_per_100k']
save_vs = ['location','population']
check = ['total_cases_per_million','new_cases_per_million', 'total_deaths_per_million','new_deaths_per_million','total_tests_per_thousand','new_tests_per_thousand','stringency_index']  

In [0]:
gini_access = {}
gf = open('/content/drive/My Drive/NASA - Integrated Assessment/cov data/combined_gini_access.csv', "r")
reader = csv.reader(gf, delimiter=',')
for ds in reader:
  gini_access[ds[0].lower()] = [ds[1],ds[2]]

defaults = {'population_density': 14.7, 'aged_65_older': 20, 'gdp_per_capita': 18381, 'hospital_beds_per_100k': 0, 'gini': 38, 'access': 442}

In [0]:
country_data = pd.read_csv(dir_path+"Italy.csv", header = None)
country_data.columns = header

In [0]:
single_use = {}
for s in single_vs:
  single_use[s] = country_data[s][0]

save_data = {}
for s in save_vs:
  save_data[s] = country_data[s][0]

drop = []
drop.extend(drop_vs)
drop.extend(single_vs)
drop.extend(save_vs)
print(drop)

country_data = country_data.drop(columns=drop)
country_data = country_data.fillna(0)

['iso_code', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_tests', 'new_tests', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_units', 'median_age', 'aged_70_older', 'cvd_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'extreme_poverty', 'population_density', 'aged_65_older', 'gdp_per_capita', 'hospital_beds_per_100k', 'location', 'population']


In [0]:
from sklearn import linear_model
n_samples = len(country_data.index)-1

X = []  
for k in range(n_samples):
  ap = []
  for dat in check:
    inpd = country_data[dat][k]
    if inpd == 0:
      inpd = 0.00001
    ap.append(inpd)
  for dat in sud:
    if single_use[dat] != 0:
      ap.append(single_use[dat])
    else:
      ap.append(default[dat])
  ga = []
  try:
    ga = [float(gini_access[save_data['location'].lower()][0]), float(gini_access[save_data['location'].lower()][1])]
  except:
    ga = [float(default['gini']), float(default['access'])]
  ap.extend(ga)
  X.append(ap)

Y = []
for k in range(1,n_samples+1):
  Y.append(country_data['new_cases_per_million'][k])

reg = linear_model.BayesianRidge()

reg.fit(X, Y)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [0]:
X = []  
for k in range(n_samples,n_samples+1):
  ap = []
  for dat in check:
    inpd = country_data[dat][k]
    if inpd == 0:
      inpd = 0.00001
    ap.append(inpd)
  for dat in sud:
    if single_use[dat] != 0:
      ap.append(single_use[dat])
    else:
      ap.append(default[dat])
  ga = []
  try:
    ga = [float(gini_access[save_data['location'].lower()][0]), float(gini_access[save_data['location'].lower()][1])]
  except:
    ga = [float(default['gini']), float(default['access'])]
  ap.extend(ga)
  X.append(ap)
print(X)
reg.predict(X)

[[3841.2340000000004, 8.533999999999999, 549.586, 1.439, 63.257, 1.147, 71.3, 205.859, 23.021, 35220.084, 3.18, 31.9, 23.33100874]]


array([10.07013277])

In [0]:
reg.coef_

array([ 8.56431117e-02,  5.66626458e-01, -6.57467641e-01,  6.96146639e-01,
        5.02590910e-01,  3.98439349e-01,  6.92396107e-02,  7.02798859e-27,
        1.38624616e-27, -2.56410553e-24,  1.40933819e-28,  0.00000000e+00,
        3.39732889e-28])